# Load Config & Initialize Paths

In [34]:
import tomllib as toml
import os
from pprint import pprint

# Load config
config_path = "Y:\\LabMembers\\MTillman\\GitRepos\\Stroke-R01\\src\\stats\\stats_config.toml"
with open(config_path, 'rb') as f:
    config = toml.load(f)
pprint(config)

# Create paths to save files
analysis_name = config["analysis_name"]
analysis_output_folder = os.path.join(config['paths']["root_save"], analysis_name)
if not os.path.exists(analysis_output_folder):
    os.makedirs(analysis_output_folder)
    print(f"Created folder: {analysis_output_folder}")

{'all_factor_columns': ['Subject',
                        'Intervention',
                        'SessionOrder',
                        'Is_Stim',
                        'Frequency',
                        'Intensity',
                        'Speed',
                        'Trial',
                        'Cycle',
                        'Side'],
 'analysis_name': 'Subject_Intervention_Speed',
 'factor_levels_to_keep': {},
 'factor_levels_to_remove': {},
 'factors_levels_order': {'Frequency': ['30', '50'],
                          'Intensity': ['RMT', 'TOL'],
                          'Intervention': ['SHAM1',
                                           'SHAM2',
                                           'RMT30',
                                           'RMT50',
                                           'TOL30',
                                           'TOL50'],
                          'Is_Stim': ['STIM', 'NO_STIM'],
                          'SessionOrder': [1, 2, 3, 4, 

# Load Data

In [35]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(config['paths']['data_file'])

pprint(config['all_factor_columns'])

# Make sure the statistical factors are pd.Categorical
for factor in config['all_factor_columns']:
    if factor in config['factors_levels_order']:
        df[factor] = pd.Categorical(df[factor], 
                                        categories=config['factors_levels_order'][factor], 
                                        ordered=True)
    else:
        df[factor] = pd.Categorical(df[factor])

# Get the list of outcome measure column names
outcome_measures_cols = [col for col in df.columns if col not in config['all_factor_columns']]

# Make sure the outcome measures are numeric
for col in outcome_measures_cols:
    df[col] = pd.to_numeric(df[col])

df.head()

['Subject',
 'Intervention',
 'SessionOrder',
 'Is_Stim',
 'Frequency',
 'Intensity',
 'Speed',
 'Trial',
 'Cycle',
 'Side']


,Subject,Intervention,SessionOrder,Is_Stim,Frequency,Intensity,Speed,Trial,Cycle,Side,...,StepDurations_Sym_PercDiff,StrideDurations_Sym_PercDiff,StepLengths_Sym_Diff,SwingDurations_Sym_Diff,StrideLengths_Sym_Diff,StanceDurations_Sym_Diff,StepWidths_Sym_Diff,StrideWidths_Sym_Diff,StepDurations_Sym_Diff,StrideDurations_Sym_Diff
0,SS01,RMT30,5,STIM,30,RMT,FV,1,1,U,...,9.662871,-0.229726,0.049186,0.073628,0.013025,-0.040075,0.031695,-0.018257,0.074016,-0.002270
1,SS01,RMT30,5,STIM,30,RMT,FV,1,2,A,...,-3.505756,-3.014979,0.018469,-0.044208,0.000087,-0.065018,0.037054,0.011232,-0.026854,-0.029792
2,SS01,RMT30,5,STIM,30,RMT,FV,1,3,U,...,-6.424463,-0.352291,0.030835,-0.096324,0.005565,0.003177,0.038754,0.011661,-0.049210,-0.003481
3,SS01,RMT30,5,STIM,30,RMT,FV,1,4,A,...,-3.705367,0.373747,0.052697,-0.081463,0.003340,0.008514,0.056866,0.013976,-0.028383,0.003693
4,SS01,RMT30,5,STIM,30,RMT,FV,1,5,U,...,-3.705367,1.201761,0.015919,-0.020916,-0.008326,0.027360,0.026810,-0.029810,-0.028383,0.011875


# Process the Data

In [ ]:
import sys
import os
current_dir = os.path.dirname(os.path.abspath("__file__"))
sys.path.append(current_dir)
from curr_col_data import curr_col_data
from make_lmer_model import make_lmer_model
for col in outcome_measures_cols:
    # Create a data frame for the current column
    curr_df = curr_col_data(df, col, config['all_factor_columns'])

    # Create the lmer model
    whole_formula = f"{col} {config['stats']['lmer']['formula']}"
    lmer_model = make_lmer_model(curr_df, whole_formula, config['stats']['lmer']['random_effect_factor'])

    # Get the estimated marginal means

    # Perform the hypothesis testing
    

['C:\\Program '
 'Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.13_3.13.752.0_x64__qbz5n2kfra8p0\\python313.zip',
 'C:\\Program '
 'Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.13_3.13.752.0_x64__qbz5n2kfra8p0\\DLLs',
 'C:\\Program '
 'Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.13_3.13.752.0_x64__qbz5n2kfra8p0\\Lib',
 'C:\\Program '
 'Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.13_3.13.752.0_x64__qbz5n2kfra8p0',
 'y:\\LabMembers\\MTillman\\GitRepos\\Stroke-R01\\.venv',
 '',
 'y:\\LabMembers\\MTillman\\GitRepos\\Stroke-R01\\.venv\\Lib\\site-packages',
 'y:\\LabMembers\\MTillman\\GitRepos\\Stroke-R01\\.venv\\Lib\\site-packages\\win32',
 'y:\\LabMembers\\MTillman\\GitRepos\\Stroke-R01\\.venv\\Lib\\site-packages\\win32\\lib',
 'y:\\LabMembers\\MTillman\\GitRepos\\Stroke-R01\\.venv\\Lib\\site-packages\\Pythonwin',
 'y:\\LabMembers\\MTillman\\GitRepos\\Stroke-R01\\src\\stats',
 'y:\\LabMembers\\MTillman\\GitRepos\\Stroke-R01\\src\\stats']
